Image Data불러오기 시행착오


In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import to_categorical

from keras.models import Sequential, load_model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
np.random.seed(1)

image    
[food] : 10000  
[interior] : 10000  
[exterior] : 10000   

----
데이터 30,000개 불러오기 실패


In [2]:
def image_load_data():
    train_images = []       
    train_labels = []
    shape = (300,300)  
    train_path = './image'

    for classes in ['f','i','e']:
        for filename in os.listdir('./image'):
            if classes == filename[0]:
                img = cv2.imread(os.path.join(train_path,filename))
                #print(filename)
                # print(img)
                # Spliting file names and storing the labels for image in list
                train_labels.append(filename[2:0:-1])

                # Resize all images to a specific shape
                #img = cv2.resize(img,shape)

                train_images.append(img)
            else:
                pass

    # Converting labels into One Hot encoded sparse matrix

    # 음식, 실내 ,실외 순으로 들어가기 위해 index 처리
    # train_labels = sorted(train_labels)
    # print(train_labels)

    train_labels = pd.get_dummies((train_labels)).values
    # print(train_labels)
    # Converting train_images to array
    train_images = np.array(train_images)

    # Splitting Training data into train and validation dataset

    train_images ,train_labels= shuffle(train_images ,train_labels, random_state = 42) 

    x_train,x_test,y_train,y_test = train_test_split(train_images,train_labels,test_size=0.2,random_state=42)
    # print(train_labels.shape)
    # print(train_labels)
    # print(len(x_train),len(x_test),len(y_train),len(y_test))

    return x_train, x_test, y_train, y_test 


활성함수  
1) Conv2D(32, kernel_size=3, activation='selu)  
2) Dropout(0.5)  
3) MaxPooling2D(pool_size=2)  
  
4) Conv2D(32, kernel_size=3, activation='selu')  
5) MaxPooling2D(pool_size=2)  
  
6) Dense(20,activation='selu')  
7) Flatten()  

In [7]:
X_train,X_test,y_train, y_test =image_load_data()
model, history = train_cnn_model()

MemoryError: Unable to allocate 3.77 GiB for an array with shape (15000, 300, 300, 3) and data type uint8

---
